In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot
from math import sqrt
from pytz import timezone

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras import optimizers

Using TensorFlow backend.


In [3]:
import talib

In [4]:
# Fix the random seed to reproducibility
np.random.seed(7)

In [5]:
lag = 20

In [6]:
def get_X_data():
    import dovahkiin as dk
    dp = dk.DataParser()
    X = dp.get_data("cu")
    return X

In [7]:
X = get_X_data()
X = pd.DataFrame({"close": X["close"]})
y = pd.Series(talib.SMA(X["close"].values, lag), index=X.index)

In [8]:
X = X["2012":]
y = y["2012":]

In [9]:
# Use difference in X as a predicator
# X = X.diff()
X = X.fillna(0)
y = y.fillna(0)

In [10]:
def timeseries_to_supervised(raw_time_series, lag):
    p = {}
    for i in range(1, lag+1):
        p["{}".format(i)] = raw_time_series.shift(i).fillna(0)
    p["0"] = raw_time_series
    
    if type(raw_time_series) is pd.Series:
        supervised_data = pd.DataFrame(p)
        supervised_data = pd.Panel({"0": supervised_data})
        supervised_data = supervised_data.swapaxes(0, 1).swapaxes(1, 2)
    else:
        supervised_data = pd.Panel(p)
    return supervised_data

In [11]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [12]:
def create_supervised_X(raw_time_series, lag):
    supervised_X = timeseries_to_supervised(X, lag)
    if not type(raw_time_series) is pd.Series:
        swaped_supervised_X = supervised_X.swapaxes(0, 1)
    else:
        swaped_supervised_X = supervised_X
    return swaped_supervised_X

In [13]:
supervised_X = create_supervised_X(X, lag=lag)
supervised_X = supervised_X.fillna(0)
supervised_X_values = supervised_X.values

In [14]:
supervised_X.shape

(500310, 21, 1)

In [15]:
supervised_X.items[350218:][0]

Timestamp('2016-03-01 13:28:00+0800', tz='Asia/Shanghai')

In [16]:
X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.3)

In [17]:
X_train = X_train[218:]
y_train = y_train[218:]
print(len(X_train))

350000


In [18]:
X_test = X_test[92:]
y_test = y_test[92:]
print(len(X_test))

150000


In [19]:
def scale(X_train, X_test):
    # scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
    scaler = Normalizer()
    # scaler = RobustScaler()
    scaler = scaler.fit(X_train)
    
    if type(X_train) is pd.Series:
        train_scaled = pd.Series(scaler.transform(X_train), index=X_train.index)
        test_scaled = pd.Series(scaler.transform(X_test), index=X_test.index)
    else:
        train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
        test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    return scaler, train_scaled, test_scaled

In [20]:
def inverse_scale(scaler, X_train_scaled, X_test_scaled):
    X_train = scaler.inverse_transform(X_train_scaled)
    X_test = scaler.inverse_transform(X_test_scaled)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train_scaled.index, columns=X_train_scaled.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test_scaled.index, columns=X_test_scaled.columns)
    return scaler, X_train, X_test

In [21]:
time_series_step = lag
features = X_train.shape[1]
batch_size = 1000

In [22]:
scaler, X_train_scaled, X_test_scaled = scale(X_train, X_test)

In [23]:
X_train_supervised_scaled = timeseries_to_supervised(X_train_scaled, time_series_step)
X_train_supervised_scaled = X_train_supervised_scaled.swapaxes(0, 1)

X_test_supervised_scaled = timeseries_to_supervised(X_test_scaled, time_series_step)
X_test_supervised_scaled = X_test_supervised_scaled.swapaxes(0, 1)

In [24]:
yscaler = MinMaxScaler()
yscaler.fit(y_train)
y_train_scaled = yscaler.transform(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [25]:
def LSTM_Model():
    model = Sequential()
    model.add(
        LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
         return_sequences=True, 
         activation="relu"
        ))
    model.add(LSTM(32, activation="relu", stateful=True))
    model.add(Dense(128))
    model.add(Dense(1))
    
    return model

In [26]:
def FullyConnected_Model():
    model = Sequential()
    model.add(Dense(128, input_shape=(time_series_step+1,)))
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [27]:
adam = optimizers.adam(lr=1e-6)

In [28]:
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=1.)

In [29]:
model = LSTM_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

In [30]:
model = FullyConnected_Model()
model.compile(loss="mean_squared_error", optimizer=adam, metrics=[ 'mse'])

In [31]:
def trainFullyConnected_network():
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            X_train_supervised_scaled.values[:, :, 0],
            y_train,
            epochs=1000,
            batch_size=1000,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )

In [32]:
trainFullyConnected_network()

Train on 280000 samples, validate on 70000 samples
Epoch 1/1000
280000/280000 [==============================] - 1s - loss: 2514040839.7714 - mean_squared_error: 2514040839.7714 - val_loss: 1402506106.5143 - val_mean_squared_error: 1402506106.5143
Epoch 2/1000
280000/280000 [==============================] - 1s - loss: 2514008616.6857 - mean_squared_error: 2514008616.6857 - val_loss: 1402481287.3143 - val_mean_squared_error: 1402481287.3143
Epoch 3/1000
280000/280000 [==============================] - 1s - loss: 2513975203.2000 - mean_squared_error: 2513975203.2000 - val_loss: 1402456148.1143 - val_mean_squared_error: 1402456148.1143
Epoch 4/1000
280000/280000 [==============================] - 1s - loss: 2513941241.6000 - mean_squared_error: 2513941241.6000 - val_loss: 1402430142.1714 - val_mean_squared_error: 1402430142.1714
Epoch 5/1000
280000/280000 [==============================] - 1s - loss: 2513906542.6286 - mean_squared_error: 2513906542.6286 - val_loss: 1402403849.1429 - val_

280000/280000 [==============================] - 1s - loss: 2511856586.9714 - mean_squared_error: 2511856586.9714 - val_loss: 1400848235.8857 - val_mean_squared_error: 1400848235.8857
Epoch 43/1000
280000/280000 [==============================] - 0s - loss: 2511773311.0857 - mean_squared_error: 2511773311.0857 - val_loss: 1400785157.4857 - val_mean_squared_error: 1400785157.4857
Epoch 44/1000
280000/280000 [==============================] - 0s - loss: 2511688353.3714 - mean_squared_error: 2511688353.3714 - val_loss: 1400720650.9714 - val_mean_squared_error: 1400720650.9714
Epoch 45/1000
280000/280000 [==============================] - 1s - loss: 2511601504.9143 - mean_squared_error: 2511601504.9143 - val_loss: 1400654981.4857 - val_mean_squared_error: 1400654981.4857
Epoch 46/1000
280000/280000 [==============================] - 1s - loss: 2511512805.9429 - mean_squared_error: 2511512805.9429 - val_loss: 1400587810.7429 - val_mean_squared_error: 1400587810.7429
Epoch 47/1000
280000/280

280000/280000 [==============================] - 0s - loss: 2506694117.9429 - mean_squared_error: 2506694117.9429 - val_loss: 1396941171.2000 - val_mean_squared_error: 1396941171.2000
Epoch 84/1000
280000/280000 [==============================] - 0s - loss: 2506516018.2857 - mean_squared_error: 2506516018.2857 - val_loss: 1396806484.1143 - val_mean_squared_error: 1396806484.1143
Epoch 85/1000
280000/280000 [==============================] - 0s - loss: 2506334988.3429 - mean_squared_error: 2506334988.3429 - val_loss: 1396669719.7714 - val_mean_squared_error: 1396669719.7714
Epoch 86/1000
280000/280000 [==============================] - 0s - loss: 2506151158.8571 - mean_squared_error: 2506151158.8571 - val_loss: 1396530658.7429 - val_mean_squared_error: 1396530658.7429
Epoch 87/1000
280000/280000 [==============================] - 0s - loss: 2505964366.6286 - mean_squared_error: 2505964366.6286 - val_loss: 1396389513.1429 - val_mean_squared_error: 1396389513.1429
Epoch 88/1000
280000/280

280000/280000 [==============================] - 0s - loss: 2496749306.9714 - mean_squared_error: 2496749306.9714 - val_loss: 1389434421.0286 - val_mean_squared_error: 1389434421.0286
Epoch 125/1000
280000/280000 [==============================] - 0s - loss: 2496430887.3143 - mean_squared_error: 2496430887.3143 - val_loss: 1389194252.8000 - val_mean_squared_error: 1389194252.8000
Epoch 126/1000
280000/280000 [==============================] - 0s - loss: 2496108446.6286 - mean_squared_error: 2496108446.6286 - val_loss: 1388951188.1143 - val_mean_squared_error: 1388951188.1143
Epoch 127/1000
280000/280000 [==============================] - 0s - loss: 2495781894.8571 - mean_squared_error: 2495781894.8571 - val_loss: 1388704877.7143 - val_mean_squared_error: 1388704877.7143
Epoch 128/1000
280000/280000 [==============================] - 0s - loss: 2495451288.2286 - mean_squared_error: 2495451288.2286 - val_loss: 1388455455.0857 - val_mean_squared_error: 1388455455.0857
Epoch 129/1000
28000

280000/280000 [==============================] - 0s - loss: 2480073798.4000 - mean_squared_error: 2480073798.4000 - val_loss: 1376875362.7429 - val_mean_squared_error: 1376875362.7429
Epoch 166/1000
280000/280000 [==============================] - 0s - loss: 2479565659.4286 - mean_squared_error: 2479565659.4286 - val_loss: 1376492999.3143 - val_mean_squared_error: 1376492999.3143
Epoch 167/1000
280000/280000 [==============================] - 0s - loss: 2479052216.2286 - mean_squared_error: 2479052216.2286 - val_loss: 1376106934.8571 - val_mean_squared_error: 1376106934.8571
Epoch 168/1000
280000/280000 [==============================] - 0s - loss: 2478533517.7143 - mean_squared_error: 2478533517.7143 - val_loss: 1375716840.2286 - val_mean_squared_error: 1375716840.2286
Epoch 169/1000
280000/280000 [==============================] - 1s - loss: 2478009476.1143 - mean_squared_error: 2478009476.1143 - val_loss: 1375322463.0857 - val_mean_squared_error: 1375322463.0857
Epoch 170/1000
28000

...

280000/280000 [==============================] - 1s - loss: 204969104.7929 - mean_squared_error: 204969104.7929 - val_loss: 4487232.7864 - val_mean_squared_error: 4487232.7864
Epoch 825/1000
280000/280000 [==============================] - 0s - loss: 201403818.0625 - mean_squared_error: 201403818.0625 - val_loss: 4300689.0062 - val_mean_squared_error: 4300689.0062
Epoch 826/1000
280000/280000 [==============================] - 1s - loss: 197871603.3321 - mean_squared_error: 197871603.3321 - val_loss: 4150402.9629 - val_mean_squared_error: 4150402.9629
Epoch 827/1000
280000/280000 [==============================] - 0s - loss: 194372004.3196 - mean_squared_error: 194372004.3196 - val_loss: 4036531.7100 - val_mean_squared_error: 4036531.7100
Epoch 828/1000
280000/280000 [==============================] - 0s - loss: 190905614.3304 - mean_squared_error: 190905614.3304 - val_loss: 3959162.0379 - val_mean_squared_error: 3959162.0379
Epoch 829/1000
280000/280000 [==============================

280000/280000 [==============================] - 1s - loss: 86655787.3631 - mean_squared_error: 86655787.3631 - val_loss: 27887738.0107 - val_mean_squared_error: 27887738.0107
Epoch 867/1000
280000/280000 [==============================] - 0s - loss: 84688144.1219 - mean_squared_error: 84688144.1219 - val_loss: 29176523.0643 - val_mean_squared_error: 29176523.0643
Epoch 868/1000
280000/280000 [==============================] - 1s - loss: 82761563.9087 - mean_squared_error: 82761563.9087 - val_loss: 30492880.3893 - val_mean_squared_error: 30492880.3893
Epoch 869/1000
280000/280000 [==============================] - 1s - loss: 80876913.9588 - mean_squared_error: 80876913.9588 - val_loss: 31837422.0607 - val_mean_squared_error: 31837422.0607
Epoch 870/1000
280000/280000 [==============================] - 1s - loss: 79032905.5346 - mean_squared_error: 79032905.5346 - val_loss: 33208369.2750 - val_mean_squared_error: 33208369.2750
Epoch 871/1000
280000/280000 [==============================

280000/280000 [==============================] - 1s - loss: 37270618.8554 - mean_squared_error: 37270618.8554 - val_loss: 94767364.5143 - val_mean_squared_error: 94767364.5143
Epoch 909/1000
280000/280000 [==============================] - 1s - loss: 36791666.0532 - mean_squared_error: 36791666.0532 - val_loss: 96312749.3714 - val_mean_squared_error: 96312749.3714
Epoch 910/1000
280000/280000 [==============================] - 1s - loss: 36336192.1508 - mean_squared_error: 36336192.1508 - val_loss: 97839442.1714 - val_mean_squared_error: 97839442.1714
Epoch 911/1000
280000/280000 [==============================] - 0s - loss: 35902501.0052 - mean_squared_error: 35902501.0052 - val_loss: 99346507.4286 - val_mean_squared_error: 99346507.4286
Epoch 912/1000
280000/280000 [==============================] - 0s - loss: 35489750.4554 - mean_squared_error: 35489750.4554 - val_loss: 100831338.9143 - val_mean_squared_error: 100831338.9143
Epoch 913/1000
280000/280000 [============================

280000/280000 [==============================] - 1s - loss: 29152830.4008 - mean_squared_error: 29152830.4008 - val_loss: 139048707.8286 - val_mean_squared_error: 139048707.8286
Epoch 951/1000
280000/280000 [==============================] - 1s - loss: 29110867.4896 - mean_squared_error: 29110867.4896 - val_loss: 139616014.4571 - val_mean_squared_error: 139616014.4571
Epoch 952/1000
280000/280000 [==============================] - 1s - loss: 29071667.1808 - mean_squared_error: 29071667.1808 - val_loss: 140166453.2571 - val_mean_squared_error: 140166453.2571
Epoch 953/1000
280000/280000 [==============================] - 1s - loss: 29034568.7544 - mean_squared_error: 29034568.7544 - val_loss: 140697074.9143 - val_mean_squared_error: 140697074.9143
Epoch 954/1000
280000/280000 [==============================] - 1s - loss: 29000426.5134 - mean_squared_error: 29000426.5134 - val_loss: 141214904.3429 - val_mean_squared_error: 141214904.3429
Epoch 955/1000
280000/280000 [====================

280000/280000 [==============================] - 1s - loss: 28505460.3943 - mean_squared_error: 28505460.3943 - val_loss: 152284674.0571 - val_mean_squared_error: 152284674.0571
Epoch 993/1000
280000/280000 [==============================] - 1s - loss: 28502213.8247 - mean_squared_error: 28502213.8247 - val_loss: 152425547.6571 - val_mean_squared_error: 152425547.6571
Epoch 994/1000
280000/280000 [==============================] - 1s - loss: 28499014.3157 - mean_squared_error: 28499014.3157 - val_loss: 152561345.7143 - val_mean_squared_error: 152561345.7143
Epoch 995/1000
280000/280000 [==============================] - 1s - loss: 28496001.1992 - mean_squared_error: 28496001.1992 - val_loss: 152692538.2857 - val_mean_squared_error: 152692538.2857
Epoch 996/1000
280000/280000 [==============================] - 1s - loss: 28493307.3714 - mean_squared_error: 28493307.3714 - val_loss: 152820545.0286 - val_mean_squared_error: 152820545.0286
Epoch 997/1000
280000/280000 [====================

### Use Raw Data to train moving average

In [31]:
raw_X_train = X_train.copy()

In [32]:
for i in range(1, lag+1):
    raw_X_train["{}".format(i)] = X_train.shift(i).fillna(0)

In [33]:
std_scaler = StandardScaler()
y_std_scaler = StandardScaler()

In [34]:
std_scaler.fit(raw_X_train)
y_std_scaler.fit(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [35]:
raw_X_train_scaled = std_scaler.transform(raw_X_train)

In [36]:
raw_y_train_scaled = y_std_scaler.transform(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [37]:
len(raw_X_train_scaled) == len(raw_y_train_scaled)

True

In [38]:
sgd = optimizers.SGD(lr=1e-7, decay=1e-8, momentum=0.9, nesterov=True)
adam = optimizers.adam(lr=1e-6)

In [39]:
adadelta = optimizers.Adadelta(lr=1e-7)

In [40]:
model = FullyConnected_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

In [41]:
def trainFullyConnected_network():   
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=120,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
        
    return model

In [42]:
model = trainFullyConnected_network()

Train on 280000 samples, validate on 70000 samples
Epoch 1/120
280000/280000 [==============================] - 1s - loss: 0.7463 - mean_squared_error: 0.7463 - val_loss: 1.9695 - val_mean_squared_error: 1.9695
Epoch 2/120
280000/280000 [==============================] - 0s - loss: 0.5900 - mean_squared_error: 0.5900 - val_loss: 1.5663 - val_mean_squared_error: 1.5663
Epoch 3/120
280000/280000 [==============================] - 0s - loss: 0.4662 - mean_squared_error: 0.4662 - val_loss: 1.2466 - val_mean_squared_error: 1.2466
Epoch 4/120
280000/280000 [==============================] - 0s - loss: 0.3684 - mean_squared_error: 0.3684 - val_loss: 0.9931 - val_mean_squared_error: 0.9931
Epoch 5/120
280000/280000 [==============================] - 0s - loss: 0.2912 - mean_squared_error: 0.2912 - val_loss: 0.7919 - val_mean_squared_error: 0.7919
Epoch 6/120
280000/280000 [==============================] - 0s - loss: 0.2302 - mean_squared_error: 0.2302 - val_loss: 0.6321 - val_mean_squared_err

280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 6.1016e-04 - val_mean_squared_error: 6.1016e-04
Epoch 101/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 6.0386e-04 - val_mean_squared_error: 6.0386e-04
Epoch 102/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.9765e-04 - val_mean_squared_error: 5.9765e-04
Epoch 103/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.9153e-04 - val_mean_squared_error: 5.9153e-04
Epoch 104/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.8551e-04 - val_mean_squared_error: 5.8551e-04
Epoch 105/120
280000/280000 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 5.7958e-04 - val_mean_squared_error: 5.795

In [43]:
raw_X_test = X_test.copy()
for i in range(1, lag+1):
    raw_X_test["{}".format(i)] = X_test.shift(i).fillna(0)

In [44]:
raw_X_test_scaled = std_scaler.transform(raw_X_test)

In [45]:
predicated_y_test_scaled = model.predict(raw_X_test_scaled)

In [53]:
predicated_y = predicated_y.reshape(150000,)

In [54]:
y_test.shape

(150000,)

In [48]:
predicated_y = y_std_scaler.inverse_transform(predicated_y_test_scaled)

In [55]:
y_diff = predicated_y - y_test

In [57]:
y_diff.abs().sum() / y_test.shape[0]

127.79531025035769

In [ ]:
y_test_scaled = y_std_scaler.transform(y_test)

In [ ]:
model.evaluate(raw_X_test_scaled, y_test_scaled)

In [ ]:
model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=20,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )

In [ ]:
model.evaluate(raw_X_test_scaled, y_test_scaled)

In [ ]:
y_test_predicated_scaled = model.predict(raw_X_test_scaled)

In [ ]:
y_test_predicated = y_std_scaler.inverse_transform(y_test_predicated_scaled)

In [ ]:
y_test_predicated

In [ ]:
len(y_test) == len(y_test_predicated)

### Use Raw Data to Train Moving Average LSTM

In [ ]:
raw_X_train = X_train.copy()
for i in range(1, lag+1):
    raw_X_train["{}".format(i)] = X_train.shift(i).fillna(0)

X_std_scaler = StandardScaler()
y_std_scaler = StandardScaler()

X_std_scaler.fit(raw_X_train)
y_std_scaler.fit(y_train)

raw_X_train_scaled = X_std_scaler.transform(raw_X_train)
raw_y_train_scaled = y_std_scaler.transform(y_train)

In [ ]:
raw_X_test = X_test.copy()
for i in range(1, lag+1):
    raw_X_test["{}".format(i)] = X_test.shift(i).fillna(0)

raw_X_test_scaled = X_std_scaler.transform(raw_X_test)
raw_y_test_scaled = y_std_scaler.transform(y_train)

In [ ]:
raw_X_train_scaled = raw_X_train_scaled.reshape(raw_X_train_scaled.shape[0], time_series_step+1, 1)
raw_X_test_scaled = raw_X_test_scaled.reshape(raw_X_test_scaled.shape[0], time_series_step+1, 1)

In [ ]:
def LSTM_Model(lstm_layers=None, dense_layers=None):
    model = Sequential()
    
    if lstm_layers:
        for i in range(lstm_layers):
            model.add(
                LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
                 return_sequences=True, 
                 activation="relu"))
        model.add(LSTM(32, activation="relu", stateful=True))
    else:
        model.add(
            LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
             return_sequences=True, 
             activation="relu"
            ))
        model.add(LSTM(32, activation="relu", stateful=True))
        
    if dense_layers:
        for i in range(dense_layers):
            model.add(Dense(128, activation="sigmoid"))
        model.add(Dense(1))
    else:
        model.add(Dense(128))
        model.add(Dense(1))
    
    return model

In [ ]:
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=1.)

In [ ]:
adam = optimizers.adam(lr=1e-6)

In [ ]:
def trainLSTM_network():   
    model = LSTM_Model(6, 3)
    model.compile(loss="mean_squared_error", optimizer=adam, metrics=[ 'mse'])
    
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=1,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
    return model

In [ ]:
model = trainLSTM_network()

In [ ]:
model.predict(raw_X_test_scaled)

In [ ]:
model.predict(raw_X_train_scaled, batch_size=batch_size)
y_predicated_scaled = model.predict(raw_X_test_scaled, batch_size=batch_size)

In [ ]:
y_predicatted = y_std_scaler.inverse_transform(y_predicated_scaled)

In [ ]:
y_predicatted

In [ ]:
y_predicated_scaled

In [ ]:
model.reset_states()

In [ ]:
y_predicated_scaled = model.predict(raw_X_test_scaled, batch_size=batch_size)

### Use Raw Data to Train Moving Average by ConvNet


In [ ]:
def ConvNet_Model():
    model = Sequential()
    model.add(Conv1D()
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [ ]:
def trainConv_network():   
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(raw_X_train_scaled),
            np.nan_to_num(raw_y_train_scaled),
            epochs=120,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
        
    return model